In [10]:
!python make_dataset.py
!python train_model.py

[{'len': 990, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 990,
  'n_features': 2,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]
[{'len': 10, 'name': '<cascade.data.pickler.Pickler', 'type': 'dataset'},
 {'len': 10,
  'name': '<cascade.data.range_sampler.RangeSampler',
  'type': 'dataset'},
 {'name': '<utils.utils.MakeClassificationDataset', 'type': 'dataset'}]
Figure(640x480)


The system cannot find the path specified.


{'f1': 0.9644268774703558}


The system cannot find the path specified.


In [2]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd

In [1]:
from correctness import correctness
from continuity import continuity
from contrastivity import contrastivity
from coherence import coherence

c:\xaib-experiments\venv3.8\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from xaib.explainers.feature_importance.constant_explainer import ConstantExplainer
from xaib.explainers.feature_importance.random_explainer import RandomExplainer
from xaib.explainers.feature_importance.shap_explainer import ShapExplainer
from xaib.explainers.feature_importance.lime_explainer import LimeExplainer

In [4]:
BS = 20

In [5]:
train_ds = cdd.Pickler('train_ds')

In [15]:
explainers = {    
    'const': ConstantExplainer(n_features=2, constant=1),
    'random': RandomExplainer(n_features=2),
    'shap': ShapExplainer(train_ds),
    'lime': LimeExplainer(train_ds, labels=(0, 1))
}

In [16]:
correctness(explainers, batch_size=BS)

100%|██████████| 1/1 [00:01<00:00,  1.95s/it]


In [17]:
continuity(explainers, batch_size=BS)

1it [00:00, 420.48it/s]
1it [00:00, 333.52it/s]
1it [00:00,  1.41it/s]
1it [00:05,  5.70s/it]


In [18]:
contrastivity(explainers, batch_size=BS)

100%|██████████| 1/1 [00:00<00:00,  4.97it/s]


In [19]:
coherence(explainers, batch_size=BS)

100%|██████████| 1/1 [00:02<00:00,  2.39s/it]


In [20]:
repo = ModelRepo('repo')

In [21]:
MetricViewer(repo).table

,line,num,created_at,saved,const,random,shap,lime,multiplier
0,repo\coherence,0,2022-11-30 20:21:51.244206+00:00,a few seconds after,{'coherence': {'other_disagreement': 1.2436576...,{'coherence': {'other_disagreement': 1.0107132...,{'coherence': {'other_disagreement': 1.6535776...,{'coherence': {'other_disagreement': 1.0427858...,NaN
1,repo\continuity,0,2022-11-30 20:18:01.885616+00:00,a few seconds after,{'continuity': {'small_noise_check': 0.0}},{'continuity': {'small_noise_check': 1.8942206...,{'continuity': {'small_noise_check': 0.0080858...,{'continuity': {'small_noise_check': 0.0137957...,0.01
2,repo\continuity,1,2022-11-30 20:21:43.758447+00:00,a few seconds after,{'continuity': {'small_noise_check': 0.0}},{'continuity': {'small_noise_check': 0.4055181...,{'continuity': {'small_noise_check': 0.0095286...,{'continuity': {'small_noise_check': 0.0135436...,0.01
3,repo\contrastivity,0,2022-11-30 20:18:07.240009+00:00,a few seconds after,{'contrastivity': {'label_difference': 0.0}},{'contrastivity': {'label_difference': 1.76665...,{'contrastivity': {'label_difference': 1.48100...,{'contrastivity': {'label_difference': 0.04608...,NaN
4,repo\contrastivity,1,2022-11-30 20:21:50.348815+00:00,a few seconds after,{'contrastivity': {'label_difference': 0.0}},{'contrastivity': {'label_difference': 0.62942...,{'contrastivity': {'label_difference': 1.48100...,{'contrastivity': {'label_difference': 0.04470...,NaN
5,repo\correctness,0,2022-11-29 20:10:11.999189+00:00,12 seconds after,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,NaN
6,repo\correctness,1,2022-11-30 19:58:11.593395+00:00,12 seconds after,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,NaN
7,repo\correctness,2,2022-11-30 20:01:05.668468+00:00,13 seconds after,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,NaN
8,repo\correctness,3,2022-11-30 20:17:47.855434+00:00,a few seconds after,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,NaN
9,repo\correctness,4,2022-11-30 20:21:41.197658+00:00,a few seconds after,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,{'correctness': {'parameter_randomization_chec...,NaN
